In [ ]:
import json
import os
from typing import Dict, List, Optional
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt

DB_FILE = "expenses_db.json"


class ExpenseApp:
    def __init__(self, users: Optional[List[str]] = None):
        self.users: List[str] = users or []
        self.transactions: List[Dict] = []  
        
    def load(self, path: str = DB_FILE) -> None:
        if not os.path.exists(path):
            return
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        self.users = data.get("users", [])
        self.transactions = data.get("transactions", [])

    def save(self, path: str = DB_FILE) -> None:
        data = {"users": self.users, "transactions": self.transactions}
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    def add_user(self, name: str) -> None:
        name = name.strip()
        if not name:
            raise ValueError("Empty name.")
        if name in self.users:
            print(f"{name} already exists.")
            return
        self.users.append(name)

    def remove_user(self, name: str) -> None:
        if name not in self.users:
            print(f"{name} not found.")
            return
        self.users.remove(name)
        cleaned: List[Dict] = []
        for tx in self.transactions:
            if tx["payer"] == name:
               continue
            if not tx.get("custom_split"):
                ben = [b for b in tx["beneficiaries"] if b != name]
                if not ben:
                     continue
                tx = {**tx, "beneficiaries": ben}
            else:
                
                cs = {k: v for k, v in tx["custom_split"].items() if k != name}
                if not cs:
                    continue
                tx = {**tx, "custom_split": cs}
            cleaned.append(tx)
        self.transactions = cleaned

   
    def add_expense(
        self,
        payer: str,
        beneficiaries: List[str],
        amount: float,
        description: str = "",
        custom_split: Optional[Dict[str, float]] = None,
    ) -> None:
        if payer not in self.users:
            raise ValueError("Payer must be an existing user.")
        for b in beneficiaries:
            if b not in self.users:
                raise ValueError(f"Beneficiary {b} is not a user.")

        if amount <= 0:
            raise ValueError("Amount must be positive.")

        if custom_split:
            for p in custom_split:
                if p not in beneficiaries:
                    raise ValueError(f"{p} not in beneficiaries.")
            total = round(sum(float(v) for v in custom_split.values()), 2)
            if round(amount, 2) != total:
                raise ValueError(
                    f"Custom split total {total} != amount {round(amount, 2)}."
                )

        tx = {
            "payer": payer,
            "beneficiaries": beneficiaries,
            "amount": float(amount),
            "description": description,
            "custom_split": custom_split or {},
        }
        self.transactions.append(tx)

   
    def _build_matrix(self) -> np.ndarray:
        n = len(self.users)
        M = np.zeros((n, n), dtype=float)  
        idx = {u: i for i, u in enumerate(self.users)}

        for tx in self.transactions:
            r = idx[tx["payer"]]
            if tx["custom_split"]:
                for person, share in tx["custom_split"].items():
                    c = idx[person]
                    M[r, c] += float(share)
            else:
                ben = tx["beneficiaries"]
                if not ben:
                    continue
                share = float(tx["amount"]) / len(ben)
                for person in ben:
                    c = idx[person]
                    M[r, c] += share
        return M

    def balances(self) -> Dict[str, float]:
        if not self.users:
            return {}
        M = self._build_matrix()
        paid = M.sum(axis=1)   
        owed = M.sum(axis=0)  
        net = paid - owed      
        out = {u: float(round(v, 2)) for u, v in zip(self.users, net)}
        return out

    def settlement_suggestions(self) -> List[Dict]:
        
        bal = self.balances()
        debtors = [(u, -v) for u, v in bal.items() if v < -1e-9]   
        creditors = [(u, v) for u, v in bal.items() if v > 1e-9]   

       
        debtors.sort(key=lambda x: x[1], reverse=True)
        creditors.sort(key=lambda x: x[1], reverse=True)

        i = j = 0
        steps: List[Dict] = []
        while i < len(debtors) and j < len(creditors):
            d_name, d_amt = debtors[i]
            c_name, c_amt = creditors[j]
            pay = round(min(d_amt, c_amt), 2)
            if pay > 0:
                steps.append({"from": d_name, "to": c_name, "amount": pay})
            d_amt = round(d_amt - pay, 2)
            c_amt = round(c_amt - pay, 2)
            debtors[i] = (d_name, d_amt)
            creditors[j] = (c_name, c_amt)
            if d_amt <= 1e-9:
                i += 1
            if c_amt <= 1e-9:
                j += 1
        return steps

   
    def print_users(self) -> None:
        t = PrettyTable(["#", "User"])
        for i, u in enumerate(self.users, 1):
            t.add_row([i, u])
        print(t)

    def print_history(self) -> None:
        t = PrettyTable(["#", "Payer", "Amount", "Beneficiaries", "Type", "Description"])
        for i, tx in enumerate(self.transactions, 1):
            if tx["custom_split"]:
                typ = "custom"
                btxt = ", ".join(f"{k}:{tx['custom_split'][k]}" for k in tx["custom_split"])
            else:
                typ = "equal"
                btxt = ", ".join(tx["beneficiaries"])
            t.add_row([i, tx["payer"], round(tx["amount"], 2), btxt, typ, tx["description"]])
        print(t)

    def print_balances(self) -> None:
        bal = self.balances()
        t = PrettyTable(["User", "Balance"])
        for u in self.users:
            t.add_row([u, bal.get(u, 0.0)])
        print(t)

    def print_suggestions(self) -> None:
        steps = self.settlement_suggestions()
        if not steps:
            print("No payments needed. Everyone is settled.")
            return
        t = PrettyTable(["From", "To", "Amount"])
        for s in steps:
            t.add_row([s["from"], s["to"], s["amount"]])
        print(t)

    def plot_balances(self) -> None:
        bal = self.balances()
        if not bal:
            print("No users to plot.")
            return
        names = list(self.users)
        values = [bal[u] for u in names]
        plt.figure()
        plt.bar(names, values)
        plt.axhline(0, linewidth=1)
        plt.title("User Balances (+ receive, - pay)")
        plt.ylabel("Amount")
        plt.tight_layout()
        plt.show()



def choose_list(prompt: str) -> List[str]:
    raw = input(prompt).strip()
    if not raw:
        return []
    parts = [p.strip() for p in raw.split(",")]
    return [p for p in parts if p]


def menu() -> None:
    app = ExpenseApp()
    app.load(DB_FILE)

    if not app.users:
        initial = choose_list("Enter users (comma separated, e.g., Alice,Bob,Charlie): ")
        if len(initial) < 2:
            print("Need at least two users to start.")
            return
        for u in initial:
            app.add_user(u)
        app.save(DB_FILE)

    while True:
        print("\n--- Expense Sharing ---")
        print("1) Add expense (equal split)")
        print("2) Add expense (custom split)")
        print("3) Show balances")
        print("4) Suggest payments")
        print("5) Show transaction history")
        print("6) Add user")
        print("7) Remove user")
        print("8) Plot balances")
        print("9) Save & Exit")

        try:
            choice = int(input("Select: ").strip())
        except ValueError:
            print("Please enter a number 1-9.")
            continue

        try:
            if choice == 1:
                app.print_users()
                payer = input("Payer name: ").strip()
                beneficiaries = choose_list("Beneficiaries (comma separated): ")
                amount = float(input("Amount: ").strip())
                desc = input("Description (optional): ").strip()
                if not beneficiaries:
                    print("No beneficiaries given.")
                    continue
                app.add_expense(payer, beneficiaries, amount, desc, custom_split=None)
                print("Expense added.")

            elif choice == 2:
                app.print_users()
                payer = input("Payer name: ").strip()
                beneficiaries = choose_list("Beneficiaries (comma separated): ")
                amount = float(input("Total amount: ").strip())
                desc = input("Description (optional): ").strip()
                if not beneficiaries:
                    print("No beneficiaries given.")
                    continue
                print("Enter custom shares as name:amount, comma separated (e.g., Alice:120,Bob:80)")
                raw = input("Custom split: ").strip()
                parts = [p.strip() for p in raw.split(",") if p.strip()]
                cs: Dict[str, float] = {}
                for p in parts:
                    if ":" not in p:
                        print(f"Skipping '{p}' (format should be name:amount).")
                        continue
                    nm, val = p.split(":", 1)
                    cs[nm.strip()] = float(val.strip())
                app.add_expense(payer, beneficiaries, amount, desc, custom_split=cs)
                print("Expense added (custom).")

            elif choice == 3:
                app.print_balances()

            elif choice == 4:
                app.print_suggestions()

            elif choice == 5:
                app.print_history()

            elif choice == 6:
                name = input("New user name: ").strip()
                app.add_user(name)
                print("User added.")

            elif choice == 7:
                name = input("Remove user name: ").strip()
                app.remove_user(name)
                print("User removed (and related transactions adjusted).")

            elif choice == 8:
                app.plot_balances()

            elif choice == 9:
                app.save(DB_FILE)
                print(f"Saved to {DB_FILE}. Bye.")
                break

            else:
                print("Choose 1-9.")
        except Exception as e:
            print(f"Error: {e}")


if __name__ == "__main__":
    menu()



--- Expense Sharing ---
1) Add expense (equal split)
2) Add expense (custom split)
3) Show balances
4) Suggest payments
5) Show transaction history
6) Add user
7) Remove user
8) Plot balances
9) Save & Exit
